## DS8008 NLP Final Project Work

#### --- LIBRARIES ---

In [1]:
#import all libraries

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import matplotlib.pyplot as plt

import numpy as np

#keras
#from keras.layers import Flatten
#from keras.layers.embeddings import Embedding
#from keras.preprocessing.text import one_hot
#from keras.preprocessing.sequence import pad_sequences

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#from gensim.corpora.dictionary import Dictionary

import os
import xml.etree.ElementTree as ET
import xml.etree.ElementTree
import string
import re


#### --- PARSER ---

In [2]:
def updateContent(child: xml.etree.ElementTree, content:str):
    if (child.text is not None):
        content=content+child.text+" "
    if (child.tail is not None):
        content=content+child.tail+ " "
    return content

In [3]:
# Funnction to parse regulations only
def parseRegulation(root: xml.etree.ElementTree):
    '''
      Parses regulation based xml file
      :param root: xml node of type xml.etree.ElementTree
      :returns dictionary of keys and values
    '''
    xRefXternal=list()
    content=""
    modifiedYear="NA"
    regYear="NA"
    consolidationYear="NA"
    #dt=""
    #modFlag=False
    xmlDict=dict()
    
    
    for child in root.iter():
        #print (child.tag,child.text,child.tail)
        if child.tag=="XRefExternal":
            txt=""
            if child.text is None:
                if child[0].tail is not None:
                    txt=child[0].tail
                else:
                     txt=child[0].text
            else:
                txt=child.text
            xRefXternal.append(txt)
            content=updateContent(child,content)
            #content+=txt+" "
       # get modifed date 
        elif ("ModifiedDate" in child.tag):
         
            dateTag=child[0]
            #print(date[1].text)
            modifiedYear=dateTag[0].text
        elif (child.tag=="RegistrationDate"):
            dateTag=child[0]
            #print(date[1].text)
            regYear=dateTag[0].text
        elif (child.tag=="ConsolidationDate"):
            dateTag=child[0]
            #print(date[1].text)
            consolidationYear=dateTag[0].text 
        elif (child.tag=="InstrumentNumber"):
            xmlDict["instrumentNumber"]=child.text
        elif (child.tag=="ShortTitle"):
            xmlDict["shorttitle"]=child.text
        elif (child.tag=="RegulationMaker"):
            xmlDict["regulationmaker"]=child.text
        elif (child.tag=="LongTitle"):
            xmlDict["longtitle"]=child.text 
            content+=child.text+". "
        elif (child.tag=="TitleText"):
            if child.text is None:
                if len(child)>0  and child[0].text is not None:
                    content+=child[0].text+". "
            else:
                content+=child.text+". "
       
            
        elif child.tag=="MM" or child.tag=="DD" or child.tag=="YYYY"  or child.tag=="Label":
            continue;
        elif child.tag=="Repealed":
            print (child.text)
            return None # igonre reglations with repeal keywords
            
        else:
            content=updateContent(child,content)
            #print(child.tag,child.text)
            '''if (child.text is not None):
            
                content=content+child.text+" "
            if (child.tail is not None):
                content=content+child.tail+ " "'''
                
           
    xmlDict["modifiedyear"]=modifiedYear
    xmlDict["registrationyear"]=regYear
    xmlDict["consolidationyear"]=consolidationYear
    xmlDict["xrefxternal"]=xRefXternal
    xmlDict["content"]=content
    
    
    return (xmlDict)

In [4]:
##################
#    Acts Parser
#################        

def parseAct(root: xml.etree.ElementTree):
    '''
      Parses regulation based xml file
      :param root: xml node of type xml.etree.ElementTree
      : returns dictionary of keys and values
    '''
    xRefXternal=list()
    xRefInternal=list()
    content=""
    regYear="NA"
    consolidationYear="NA"
    regnalYears="NA"
     
    #dt=""
    #modFlag=False
    xmlDict=dict()
    
    
    for child in root.iter():
        #print (child.tag,child.get("stage"))
        if child.tag=="XRefExternal":
            txt=""
            if child.text is None:
                if child[0].tail is not None:
                    txt=child[0].tail
                else:
                     txt=child[0].text
            else:
                txt=child.text
            xRefXternal.append(txt)
            #content+=txt+" "
            content=updateContent(child,content)
       # get modifed date 
       
        elif ("Stages"==child.tag and child.get("stage")=="assented-to"):
            
            dateTag=child[0]
            #print(date[1].text)
            regYear=dateTag[0].text
        elif (child.tag=="Stages" and child.get("stage")=="consolidation"):
            dateTag=child[0]
            #print(date[1].text)
            consolidationYear=dateTag[0].text
        elif (child.tag=="RegnalYear"):
            regnalYears=child.text
        elif (child.tag=="ConsolidatedNumber"):
            xmlDict["ConsolidatedNumber"]=child.text
        elif (child.tag=="ShortTitle"):
            xmlDict["shorttitle"]=child.text
        elif (child.tag=="RunningHead"):
            xmlDict["runninghead"]=child.text
        elif (child.tag=="ConsolidatedNumber"):
            xmlDict["consolidatednumber"]=child.text
        elif (child.tag=="AnnualStatuteNumber"):
            xmlDict["annualstatutenumber"]=child.text
        elif (child.tag=="XRefInternal"):
            xRefInternal.append(child.text)
        elif (child.tag=="LongTitle"):
            if (child.text is not None): # funny sometimes there are two long titles 
                xmlDict["longtitle"]=child.text 
                content+=child.text+". "
        elif (child.tag=="TitleText"):
            if child.text is None:
                if len(child)>0  and child[0].text is not None:
                    content+=child[0].text+". "
            else:
                content+=child.text+". "
       
       
            
        elif child.tag=="MM" or child.tag=="DD" or child.tag=="YYYY"  or child.tag=="Label":
            continue;
        elif child.tag=="Repealed":
            print (child.text)
            return None # igonre reglations with repeal keywords
            
            
        else:
            #print(child.tag,child.text)
            content=updateContent(child,content)
            '''if (child.text is not None):
            
                content=content+child.text+" "
            if (child.tail is not None):
                content=content+child.tail+ " "'''
                
    
    
    xmlDict["registrationyear"]=regYear
    xmlDict["consolidationyear"]=consolidationYear
    xmlDict["regnalyears"]=regnalYears
    xmlDict["xrefxternal"]=xRefXternal
    xmlDict["xrefinternal"]=xRefInternal
    xmlDict["content"]=content
    
    
    return (xmlDict)

In [5]:
# Unit test: usage method

#SOR-2009-317.xml

def get_file_from_folder (file_name):
    directory="//Users//saad//Saad//Education//Ryerson//DS8008 NLP//Project//dataset//regulations"
    #directory="..somepaths/acts"
    count=0
    n=0
    for filename in os.listdir(directory):
        if filename.endswith(".xml") and (not filename.startswith("SI-") and filename==file_name):
            #print (filename)
            path=os.path.join(directory, filename)
            tree=ET.parse(path)
            d=parseRegulation(tree.getroot())
            #print (d)
            n=n+1
    return d
#       # d=parseAct(tree.getroot())
#        #if (d==None):
#         #   print (filename)
#          #  count =count +1
#        print(d)
#print(count)
#print (n)
#       # d=parseAct(tree.getroot())
        #print(d)

#### --- WORKING CODE ---

In [6]:
##get all file names in the directory

#path to directory
path = '/Users//saad//Saad//Education//Ryerson//DS8008 NLP//Project//dataset//regulations'

files = []
for f in os.listdir(path):
    if f.endswith(".xml") and (not f.startswith("SI-")):
        files.append(f)
                               
    
print("total files in directory: %d" % len(files))

total files in directory: 3209


In [7]:
files[:5]

['SOR-2017-109.xml',
 'SOR-85-567.xml',
 'SOR-93-293.xml',
 'SOR-98-87.xml',
 'C.R.C.,_c._208.xml']

In [8]:
#store all docs in a dictionary
main_doc_dict ={}

i=0
for f_name in files:
    key_name = "doc_"+str(i)
    main_doc_dict[key_name] = get_file_from_folder(f_name)
    i+=1

[Repealed, SOR/2011-173]
[Revoked, SOR/80-90]
[Repealed, SOR/2017-94, s. 24]
[Repealed, SOR/2016-49, s. 4]
[Repealed, SOR/97-557, s. 3]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SI/2015-112]
[Repealed, SOR/2008-50, s. 6]
[Repealed, SOR/2011-6, s. 1]
[Repealed, SOR/2010-117, s. 59]
[Repealed, SOR/2006-126, s. 1]
[Revoked, SOR/92-737, s. 1]
[Repealed, SOR/98-535, s. 2]
[Repealed, SOR/81-174, s. 1]
[Repealed, SOR/2002-1, s. 2]
[Repealed, SOR/2017-91, s. 1]
[Repealed, SOR/2008-168, s. 1]
[Repealed, SOR/2013-61, s. 7]
[Repealed, SOR/2014-193, s. 1]
[Repealed, SOR/2004-109, s. 1]
[Repealed, SOR/2002-237, s. 2]
[Repealed, SOR/2006-318, s. 6]
[Repealed, SOR/2009-62, s. 1]
[Repealed, SOR/97-43, s. 2]
[Repealed, SOR/2008-163, s. 3]
[Repealed, SOR/95-25, s. 1]
[Repealed, SOR/2001-494, s. 8]
[Repealed, SOR/2009-223, s. 4]
[Repealed, SOR/2016-289, s. 3]
[Repealed, SOR/2005-355, s. 1]
[Repealed, SOR/2007-198, s. 1]
[Repealed, SOR/2016-102, s. 1]
[Repealed, SOR/2009-45, s. 4]
[Repealed, SOR/2013-254, 

[Repealed, SOR/2009-46, s. 4]
[Repealed, SOR/2017-285, s. 2]
[Repealed, SOR/2008-120, s. 22]
[Repealed, SOR/91-420, s. 1]
[Repealed, SOR/2016-319, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2013-226, s. 1]
[Repealed, SOR/2006-168, s. 3]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2016-168, s. 3]
[Repealed, SOR/97-39, s. 2]
[Repealed, SOR/2002-190, s. 1]
[Repealed, SOR/2016-203, s. 53]
[Repealed, SOR/2016-319, s. 1]
[Revoked, SOR/80-90]
[Repealed, SOR/2018-42, s. 2]
[Repealed, SOR/2009-28, s. 4]
[Repealed, SOR/2010-288, s. 5]
 [Repealed, SOR/2008-168, s. 3]
[Repealed, SI/2008-91]
[Repealed, SOR/2008-169, s. 1]
[Revoked, SOR/79-899, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Revoked, SOR/93-401, s. 2]
[Repealed, SOR/2015-203, s. 15]
[Repealed, SOR/2017-84, s. 1]
[Revoked, SOR/86-999, s. 12]
[Repealed, SOR/2005-408, s. 2]
[Repealed, SOR/2017-54, s. 1]
[Repealed, SOR/2017-84, s. 1]
 [Repealed, SOR/2008-177, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2017-110, s. 7]
[Repealed, SO

[Repealed, SOR/2007-128, s. 406]
[Repealed, SOR/97-176, s. 1]
[Repealed, SOR/2006-162, s. 3]
[Repealed, SOR/2007-86, s. 206]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2015-184, s. 1]
[Repealed, SOR/2004-299, s. 2]
[Repealed, SOR/2017-135, s. 1]
[Repealed, SOR/2011-205, s. 16]
[Repealed, SOR/2007-94, s. 3]
[Repealed by operation of law, 1997, c. 20, s. 45
[Repealed, SOR/2012-49, s. 1]
[Repealed, SOR/2007-99, s. 23]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2007-115, s. 402]
[Repealed, SOR/2013-68, s. 3]
[Repealed, SOR/2016-289, s. 12]
[Repealed, SOR/2014-153, s. 2]
[Repealed, SOR/2014-250, s. 2]
[Repealed, SOR/2008-305, s. 3]
[Revoked, SOR/80-90]
[Repealed, 1998, c. 10, s. 201, see SI/2013-107]
[Repealed, SOR/2009-231, s. 7]
[Repealed, SOR/93-232, s. 2]
[Repealed, SOR/2012-17, s. 3]
[Repealed, SOR/2017-181, s. 2]
[Repealed, SOR/2004-244, s. 1]
[Repealed, SOR/2009-322, s. 23]
[Repealed, SOR/2010-13, s. 3]
[Repealed, SOR/2006-176, s. 1]
[Repealed, SOR/2002-179, s. 1]
[Repealed, SOR/

[Repealed, SOR/2005-148, s. 2]
[Repealed, SOR/2012-124, s. 3]
[Revoked, SOR/80-90]
[Repealed, SOR/2015-72, s. 1]
[Repealed, SOR/2001-16, s. 7]
[Revoked, SOR/81-805, s. 2]
[Repealed, SOR/2016-252, s. 27]
[Repealed, SOR/2014-285, s. 1]
[Repealed, P.C. 2013-1284]
[Repealed, SOR/2017-252, s. 3]
[Repealed, SOR/2016-319, s. 1]
[Repealed, SOR/2001-179, s. 4]
[Repealed, SOR/2017-14, s. 416]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2008-317, s. 1]
[Revoked, SI/91-12, s. 2]
[Revoked, SI/83-181, s. 2]
[Repealed, SOR/2014-309, s. 1]
[Repealed, SOR/2009-296, s. 2]
[Repealed, SOR/2011-80, s. 2]
[Revoked, SOR/90-468, s. 1]
[Repealed, SOR/2011-15, s. 6]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2016-231, s. 4]
[Repealed, SOR/2008-168, s. 23]
[Repealed, SOR/97-177, s. 9]
[Repealed, SOR/2007-303, s. 44]
[Repealed, SOR/2003-302, s. 2]
[Repealed, SOR/2013-119, s. 267]
[Repealed by operation of law, 2011, c. 25, s. 39]
[Repealed, SOR/2006-319, s. 5]
[Revoked, SOR/78-842

[Repealed, SOR/2008-61, s. 2]
[Repealed, SOR/2002-166, s. 2]
[Repealed, SOR/2008-170, s. 2]
[Repealed, SOR/2007-303, s. 44]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2009-317, s. 105]
[Repealed, SOR/2014-75, s. 9]
[Repealed, SOR/2005-211, s. 1]
[Repealed, SOR/2009-95, s. 1]
[Repealed, SOR/2016-167, s. 16]
[Repealed, SOR/2001-51, s. 1]
[Repealed, SOR/2012-33, s. 3]
[Repealed, SOR/2013-19, s. 6]
[Repealed, SOR/2014-293, s. 11]
[Repealed, SOR/2012-69, s. 136]
[Repealed, SOR/2013-159, s. 1]
[Repealed, SOR/2006-319, s. 4]
[Repealed, SOR/93-335, s. 1]
[Repealed, SOR/2015-72, s. 1]
[Repealed, SOR/2017-58, s. 25]
[Repealed, SOR/2000-256, s. 1]
[Repealed, SOR/2006-320, s. 4]
[Repealed, SOR/2007-163, s. 1]
[Repealed, SOR/2014-293, s. 11]
[Repealed, SOR/2006-156, s. 1]
[Revoked, SOR/90-595, s. 1]
[Repealed, SOR/2013-166, s. 2]
[Repealed, SOR/96-278, s. 1]
[Repealed, SOR/2016-273, s. 1]
[Repealed, SOR/2016-179, s. 11]
[Repealed, SOR/2008-170, s. 1]
[Repealed, SOR/80-738, s. 1]
[Repealed, SOR/98

In [9]:
print("total files in dictionary: %d" % len(main_doc_dict))

total files in dictionary: 3209


In [10]:
#clean dict by removing none files
main_doc_dict_clean={}
for val in main_doc_dict:
    if main_doc_dict[val] is not None:
        main_doc_dict_clean[val] = main_doc_dict[val]

print("new total files in dictionary: %d" % len(main_doc_dict_clean))

new total files in dictionary: 2062


In [11]:
#get keys for the docs
main_doc_dict_clean['doc_1'].keys()

dict_keys(['instrumentNumber', 'shorttitle', 'longtitle', 'regulationmaker', 'modifiedyear', 'registrationyear', 'consolidationyear', 'xrefxternal', 'content'])

In [12]:
#create a dataframe of files
import pandas as pd

df_regulations = pd.DataFrame.from_dict(main_doc_dict_clean)
df_regulations = df_regulations.T

In [13]:
df_regulations.head(5)

,consolidationyear,content,instrumentNumber,longtitle,modifiedyear,registrationyear,regulationmaker,shorttitle,xrefxternal
doc_0,2018,ENVIRONMENTAL VIOLATIONS ADMINISTRATIVE MONETA...,SOR/2017-109,Environmental Violations Administrative Moneta...,2017,2017,P.C.,NaN,[ENVIRONMENTAL VIOLATIONS ADMINISTRATIVE MONET...
doc_1,2018,CANADA POST CORPORATION ACT Regulations Respec...,SOR/85-567,Regulations Respecting Mailable Matter Bearing...,NA,1985,P.C.,Deficient Postage Regulations,"[CANADA POST CORPORATION ACT, Canada Gazette, ..."
doc_2,2018,AERONAUTICS ACT Regulations Respecting Zoning ...,SOR/93-293,Regulations Respecting Zoning at Bagotville Ai...,2011,1993,P.C.,Bagotville Airport Zoning Regulations,"[AERONAUTICS ACT, Aeronautics Act, Regulations..."
doc_3,2018,CUSTOMS TARIFF Shirting Fabrics Remission Orde...,SOR/98-87,"Shirting Fabrics Remission Order, 1998",2005,1997,P.C.,NaN,"[CUSTOMS TARIFF, Customs Tariff, Shirting Fabr..."
doc_4,2018,AGRICULTURAL PRODUCTS MARKETING ACT Regulation...,"C.R.C., c. 208",Regulations Respecting the Marketing in Interp...,2005,NA,NaN,Ontario Greenhouse Vegetable Marketing (Interp...,"[AGRICULTURAL PRODUCTS MARKETING ACT, Ontario ..."


In [14]:
#get content from df, get index name which is equal to doc number and get titles
doc_content = df_regulations.content.values
df_index = df_regulations.index.tolist()
df_titles = df_regulations.longtitle.values

In [15]:

print("total content: %d" % len(doc_content))

total content: 2062


In [16]:
#following is the tokenizer - stop words + special words + punctuations

stop_words = stopwords.words('english')
remove_words = ['s.c.','subsection','regulation','act','set','section','paragraph','subheading','heading','chapter','column','person','respecting','respect','minister','year','date','january','type','part','division']
stop_words.extend(remove_words)
punctuationList=list(string.punctuation)
#4202.39-4202.91

In [17]:
## USING THIS ONE FOR FINAL CODE
def myTokenizer_lem(text):
    
    lemmatizer = WordNetLemmatizer()
    lemmas=[]
    #tokens = nltk.word_tokenize(text)
    
    for sent in nltk.sent_tokenize(text):
        #nltk return the tag from Penntreebank tagsets
        sentTag=nltk.pos_tag(nltk.word_tokenize(sent))
        
        for word, tag in sentTag:
            if word.isnumeric()== False and word not in punctuationList and word not in stop_words and word.isalpha()==True:
                wordNetTag=getWordnetPos(tag)
                
                if wordNetTag is None:
                    continue
                else:
                    word = word.lower()
                    lemma_word = lemmatizer.lemmatize(word,wordNetTag)
                    if len(lemma_word) > 3 and lemma_word not in remove_words:
                        lemmas.append(lemma_word)

    return lemmas
                

# Function to convert 
#Penntreebank tags to wordnet tags
def getWordnetPos(treebank_tag):
    
    if treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    #elif treebank_tag.startswith('V'):
    #    return wordnet.VERB
    else:
        return None  

In [18]:
#word='environmental_violations'
#word.isalpha()

In [19]:
#create frequent bigrams

from gensim.models.phrases import Phrases, Phraser

bigram_tokens = []
for single_doc in doc_content:
    new_list = []
    new_list.append(myTokenizer_lem(single_doc))
    #print(new_list[0])
    phrases = Phrases(new_list, min_count=1, threshold=10)  # train model
    bigram_tokens.append(phrases[new_list[0]])
    


/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [20]:
bigram_tokens[1]

['canada',
 'post_corporation',
 'mailable_matter',
 'bearing',
 'postage',
 'insufficient',
 'postage',
 'whereas',
 'copy',
 'canada',
 'gazette',
 'april',
 'reasonable',
 'opportunity',
 'thereby',
 'interested',
 'representation',
 'thereto',
 'therefore',
 'excellency',
 'governor',
 'general',
 'council',
 'recommendation',
 'national',
 'revenue',
 'pursuant',
 'canada',
 'post_corporation',
 'pleased',
 'hereby',
 'annexed',
 'mailable_matter',
 'postage',
 'insufficient',
 'postage',
 'canada',
 'post_corporation',
 'effective',
 'september',
 'short',
 'title',
 'deficient',
 'postage',
 'interpretation',
 'deficient',
 'postage',
 'amount',
 'postage',
 'payable',
 'mailable_matter',
 'affranchissement',
 'insuffisant',
 'deficient',
 'postage',
 'item',
 'item',
 'mailable_matter',
 'deficient',
 'postage',
 'envoi',
 'insuffisamment',
 'affranchi',
 'application',
 'mailable_matter',
 'canada_transmission',
 'post_delivery',
 'canada',
 'general',
 'condition',
 'subject'

In [21]:
#copying bigrams
doc_content_tokens = bigram_tokens

In [22]:
#doc_content_tokens[0]

#### --- MAIN BATCH CODE ---

In [24]:
# CODE FOR BATCH PROCESS
#get all topics in a dictionary and store coherence score
import operator

all_topics = {}
batch_coherence = []

start = 0
end =100
for i in range(21):
    if i == 20:
        common_dictionary_tfidf = corpora.Dictionary(doc_content_tokens[start:])
        common_corpus_tfidf = [common_dictionary_tfidf.doc2bow(text) for text in doc_content_tokens[start:]]
    else:
        common_dictionary_tfidf = corpora.Dictionary(doc_content_tokens[start:end])
        common_corpus_tfidf = [common_dictionary_tfidf.doc2bow(text) for text in doc_content_tokens[start:end]]
    
    # Train the Hierarchical Dirichlet Process on the corpus
    hdp = gensim.models.hdpmodel.HdpModel(corpus= common_corpus_tfidf,id2word=common_dictionary_tfidf,random_state=1)
    
    #convert hdp to lda model and then get topics by doc
    trained_lda = hdp.suggested_lda_model()
    
    if i == 20:
        cm = gensim.models.coherencemodel.CoherenceModel(trained_lda,texts = doc_content_tokens[start:],corpus=common_corpus_tfidf, coherence='c_v')
        batch_coherence.append(cm.get_coherence())
    else:
        cm = gensim.models.coherencemodel.CoherenceModel(trained_lda,texts = doc_content_tokens[start:end],corpus=common_corpus_tfidf, coherence='c_v')
        batch_coherence.append(cm.get_coherence())
        
    
    doc_dict = {}
    for index,doc in enumerate(common_corpus_tfidf):
        singel_doc_dict = {}
        topics = trained_lda.get_document_topics(doc)
        topics_sorted = sorted(topics, key = operator.itemgetter(1))
        
        #get top 2 topics only
        if len(topics_sorted) > 2:
            topics_sorted = topics_sorted[-2:]
            
        words=[]
        #get words of the topics
        for topic in topics_sorted:
            topic_num = topic[0]
            li = [hdp.print_topic(topic_num, topn=20)]
            words.append(li)
            
        singel_doc_dict['doc_num'] = df_index[start+index]
        singel_doc_dict['doc_name'] = df_titles[start+index]
        singel_doc_dict['topics'] = topics_sorted
        singel_doc_dict['words_topic'] = words
        
        doc_dict[df_index[start+index]] = singel_doc_dict
        
    name = "batch_"+str(i)
    all_topics[name] = doc_dict
    
    start = end
    end += 100
 
    

In [25]:
all_topics.keys()

dict_keys(['batch_0', 'batch_1', 'batch_2', 'batch_3', 'batch_4', 'batch_5', 'batch_6', 'batch_7', 'batch_8', 'batch_9', 'batch_10', 'batch_11', 'batch_12', 'batch_13', 'batch_14', 'batch_15', 'batch_16', 'batch_17', 'batch_18', 'batch_19', 'batch_20'])

In [26]:
all_topics['batch_0'].keys()

dict_keys(['doc_0', 'doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5', 'doc_7', 'doc_9', 'doc_13', 'doc_14', 'doc_15', 'doc_16', 'doc_17', 'doc_19', 'doc_20', 'doc_24', 'doc_26', 'doc_27', 'doc_29', 'doc_30', 'doc_31', 'doc_33', 'doc_35', 'doc_37', 'doc_38', 'doc_39', 'doc_40', 'doc_41', 'doc_43', 'doc_44', 'doc_45', 'doc_46', 'doc_51', 'doc_56', 'doc_57', 'doc_58', 'doc_60', 'doc_61', 'doc_63', 'doc_64', 'doc_65', 'doc_68', 'doc_69', 'doc_72', 'doc_74', 'doc_75', 'doc_76', 'doc_78', 'doc_82', 'doc_83', 'doc_84', 'doc_85', 'doc_87', 'doc_89', 'doc_91', 'doc_93', 'doc_95', 'doc_97', 'doc_98', 'doc_100', 'doc_101', 'doc_104', 'doc_105', 'doc_106', 'doc_109', 'doc_110', 'doc_111', 'doc_112', 'doc_115', 'doc_116', 'doc_117', 'doc_118', 'doc_122', 'doc_123', 'doc_124', 'doc_126', 'doc_127', 'doc_129', 'doc_130', 'doc_131', 'doc_134', 'doc_135', 'doc_136', 'doc_137', 'doc_138', 'doc_141', 'doc_142', 'doc_143', 'doc_144', 'doc_145', 'doc_146', 'doc_148', 'doc_151', 'doc_152', 'doc_154', 'doc_155', 

In [27]:
all_topics['batch_0']['doc_166'].keys()

dict_keys(['doc_num', 'doc_name', 'topics', 'words_topic'])

In [29]:
#batch model coherence score
print('batch coherence score %s' % (sum(batch_coherence)/len(batch_coherence)))

batch coherence score 0.7836116140981634


#### --- CREATING DF FOR EXPORT ---

In [30]:
# Dictionary with list object in values
batch_num = []
doc_num= []
doc_name=[]
topics=[]
words_topic=[]

for batch_key,batch_val in all_topics.items():
    for doc_key,doc_val in batch_val.items():
        batch_num.append(batch_key)
        doc_num.append(doc_key)
        doc_name.append(doc_val['doc_name'])
        topics.append(doc_val['topics'])
        words_topic.append(doc_val['words_topic'])
        

In [31]:
d = {'batch_num':batch_num,'doc_num':doc_num,'doc_name':doc_name,'topics':topics,'words_topic':words_topic}

In [32]:
df_topics_by_docs = pd.DataFrame.from_dict(d)
#print("df shape %d,%d" %(df_topics_by_docs[0],df_topics_by_docs[1]))
df_topics_by_docs.head(5)

,batch_num,doc_num,doc_name,topics,words_topic
0,batch_0,doc_0,Environmental Violations Administrative Moneta...,"[(17, 0.9981149665104837)]","[[0.012*""violation"" + 0.007*""item_provision"" +..."
1,batch_0,doc_1,Regulations Respecting Mailable Matter Bearing...,"[(54, 0.993009638155919)]","[[0.005*""postage"" + 0.004*""deficient"" + 0.003*..."
2,batch_0,doc_2,Regulations Respecting Zoning at Bagotville Ai...,"[(12, 0.9987329765129673)]","[[0.012*""surface"" + 0.009*""line"" + 0.006*""stri..."
3,batch_0,doc_3,"Shirting Fabrics Remission Order, 1998","[(32, 0.9973742696164394)]","[[0.010*""fabric"" + 0.006*""shirt"" + 0.006*""cust..."
4,batch_0,doc_4,Regulations Respecting the Marketing in Interp...,"[(35, 0.9948476013037094)]","[[0.008*""product"" + 0.003*""producer"" + 0.003*""..."


In [33]:
#un comment following to export to excel
#df_topics_by_docs.to_excel("//Users//saad//Desktop//topics_output_project.xlsx")

#### --- OTHER MODELS TRIED ---

In [34]:
#tfidf
from gensim.models import TfidfModel

common_dictionary_tfidf = corpora.Dictionary(doc_content_tokens)
common_corpus_tfidf = [common_dictionary_tfidf.doc2bow(text) for text in doc_content_tokens]


In [35]:
# Train the model on the corpus
lda = gensim.models.ldamodel.LdaModel(corpus= common_corpus_tfidf,id2word=common_dictionary_tfidf, num_topics=80,passes=50,alpha ='auto',iterations=200,random_state=1)

In [36]:
lda.get_document_topics(common_corpus_tfidf[0])

[(3, 0.038384523),
 (4, 0.71029377),
 (33, 0.054195922),
 (35, 0.038307823),
 (53, 0.010285926),
 (63, 0.0126164695),
 (74, 0.02461416),
 (75, 0.010471005),
 (79, 0.012960648)]

In [37]:
lda.print_topic(74, topn=20)

'0.094*"water" + 0.092*"vessel" + 0.053*"ballast" + 0.017*"certificate" + 0.015*"exchange" + 0.015*"master" + 0.014*"canadian_jurisdiction" + 0.014*"area" + 0.009*"board_vessel" + 0.009*"requirement" + 0.009*"following" + 0.008*"board" + 0.008*"exception" + 0.008*"depth_least" + 0.007*"canadian" + 0.007*"voyage" + 0.007*"terminal_anchorage" + 0.006*"least" + 0.006*"great_lake" + 0.006*"case"'

In [38]:
# Train the Hierarchical Dirichlet Process on the corpus
hdp = gensim.models.hdpmodel.HdpModel(corpus= common_corpus_tfidf,id2word=common_dictionary_tfidf,random_state=1)

In [39]:
print("total topics found by hdp: %d" % hdp.get_topics().shape[0])

total topics found by hdp: 150


In [40]:
#convert hdp to lda model and then get topics by doc
trained_lda = hdp.suggested_lda_model()



#### --- EVALUATE MODELS ---

In [41]:
cm = gensim.models.coherencemodel.CoherenceModel(lda,texts = doc_content_tokens,corpus=common_corpus_tfidf, coherence='c_v')
cm.get_coherence()

0.5221588802604662

In [42]:
cm2 = gensim.models.coherencemodel.CoherenceModel(trained_lda,texts = doc_content_tokens,corpus=common_corpus_tfidf, coherence='c_v')
cm2.get_coherence()

0.7150817734915684

In [43]:
cm3 = gensim.models.coherencemodel.CoherenceModel(hdp,texts = doc_content_tokens,corpus=common_corpus_tfidf, coherence='c_v')
cm3.get_coherence()

0.5988955888441746

In [44]:
try:
    import pyLDAvis.gensim
except ImportError:
    ValueError("SKIP: please install pyLDAvis")
    
import json
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity

In [45]:
pyLDAvis.enable_notebook()

In [46]:
pyLDAvis.gensim.prepare(hdp, common_corpus_tfidf, common_dictionary_tfidf)

PreparedData(topic_coordinates=              x         y  topics  cluster          Freq
topic                                                   
0      0.309785 -0.091549       1        1  4.207353e+01
1      0.336513 -0.130783       2        1  8.397829e+00
2      0.285944 -0.090603       3        1  8.013940e+00
3      0.306341 -0.118693       4        1  7.588518e+00
4      0.247833  0.312906       5        1  5.497018e+00
5      0.227988 -0.070967       6        1  4.636277e+00
7      0.150525  0.275608       7        1  2.426849e+00
6      0.181867  0.264141       8        1  2.039126e+00
8      0.165483 -0.084107       9        1  1.663448e+00
12     0.061502 -0.046960      10        1  1.412610e+00
16     0.075431 -0.044294      11        1  1.345904e+00
9      0.113282 -0.038495      12        1  1.325136e+00
15     0.082660 -0.054446      13        1  1.076584e+00
10     0.085216 -0.053794      14        1  1.040840e+00
11     0.106441 -0.067689      15        1  1.039919e+00
13     0.055068 -0.010518      16        1  9.522833e-01
18     0.047939  0.061982      17        1  5.733879e-01
24     0.024342 -0.028131      18        1  5.624226e-01
17     0.050137 -0.027141      19        1  5.277428e-01
36    -0.001803 -0.016516      20        1  4.247918e-01
25     0.015802  0.097649      21        1  4.081876e-01
19     0.046195 -0.032554      22        1  4.062622e-01
21     0.050038 -0.040066      23        1  3.651649e-01
23     0.022733 -0.021546      24        1  3.637453e-01
31     0.023449 -0.023665      25        1  3.414111e-01
26     0.027352 -0.024926      26        1  3.351214e-01
22    -0.010011 -0.007101      27        1  3.184538e-01
28     0.029471 -0.029609      28        1  2.664735e-01
30     0.026467 -0.019615      29        1  2.623136e-01
29    -0.010166 -0.007959      30        1  2.294658e-01
...         ...       ...     ...      ...           ...
107   -0.043497  0.002922     121        1  9.891096e-06
110   -0.042191  0.002057     122        1  6.228339e-06
113   -0.043850  0.003086     123        1  3.410613e-06
114   -0.042795  0.001757     124        1  2.736049e-06
115   -0.044080  0.002181     125        1  2.193805e-06
116   -0.045154  0.001723     126        1  1.765559e-06
117   -0.042563  0.001204     127        1  1.713938e-06
118   -0.043350  0.000777     128        1  1.112979e-06
121   -0.043242  0.001482     129        1  5.962512e-07
124   -0.042967  0.001210     130        1  2.230307e-07
125   -0.042890  0.000827     131        1  1.472740e-07
126   -0.042971  0.002081     132        1  9.702439e-08
127   -0.041190  0.000401     133        1  7.765856e-08
128   -0.044276  0.002915     134        1  4.000802e-08
130   -0.042347  0.001665     135        1  1.560866e-08
131   -0.042789  0.001195     136        1  1.022301e-08
132   -0.043253  0.001508     137        1  5.938327e-09
134   -0.043672  0.001289     138        1  2.201514e-09
137   -0.043145  0.001973     139        1  4.844742e-10
138   -0.043226  0.001855     140        1  2.904652e-10
139   -0.043490  0.002472     141        1  1.735276e-10
140   -0.042086  0.000810     142        1  1.032932e-10
141   -0.043399  0.000681     143        1  6.793158e-11
142   -0.043458  0.001131     144        1  3.501209e-11
143   -0.043058  0.001573     145        1  1.992478e-11
144   -0.044264  0.002067     146        1  1.128959e-11
146   -0.043158  0.001635     147        1  3.425529e-12
147   -0.044550  0.001415     148        1  1.832783e-12
148   -0.043411  0.001783     149        1  1.082159e-12
149   -0.045392  0.001204     150        1  9.753270e-13

[150 rows x 5 columns], topic_info=       Category          Freq                     Term        Total  loglift  \
term                                                                           
356     Default  4.279000e+03                  surface  4279.000000  30.0000   
326     Default  4.373000e+03                    point  4373.000000  29.0000   
389     Default  4.220